# Tarea 1
## Ejercicio 1
El objetivo de las bases es que realicemos un modelo de clasificación para poder contestar la siguiente pregunta: **¿Damos un crédito bancario o no?**

### Parte 1: Exploración de las bases
Para empezar con la exploración de la base, tendrán que generar un análisis descriptivo de la información que les compartiré:

 1. Los campos _id_ que son los identificadores de la información.
 2. _base_ini_interna_.
 3. _base_ini_parte1_.
 4. _base_ini_externa_.
 5. Bases de comportamiento.
 6. _base_inversión_, que es una variable simulada de una gamma.
 7. Cuando una variable es categórica, está implícito en el nombre de la base. No tenemos un diccionario, simplemente son categóricas.
 8. El resto entonces son continuas. Es decir, los nombre estám enmascarados.

Utilizaremos _sqldf_, así que antes se debe instalar de ser necesario.

In [1]:
pip install sqldf

Note: you may need to restart the kernel to use updated packages.


A continuación se cargarán las bibliotecas que serán, muy probablemente, utilizadas.

In [2]:
#import sqlite3
import sqldf
import pandas as pd
from pylab import*
import numpy as np
import scipy.stats 
from scipy.special import gamma
import seaborn as sns
from scipy.stats import beta
from scipy.stats import spearmanr
import pandas as pd

Ahora procedemos a cargar cada una de las bases del enunciado, iniciando los primeros pasos exploratorios. De manera general, se aplicará el mismo procedimiento a cada base y después, dependiendo de los datos que cada una presente, se observarán algunos tratamientos distintos. El procedimiento default corresponde a sacar, con _info()_, un resumen general de los tipos de datos que manejan las bases. Además, con sqldf se validará que no haya id's repetido si es el caso.

In [3]:
ruta = "./data/"
bd = "base_ini_externa.txt"
bd_ini_externa = pd.read_csv(ruta+bd,sep=" ")
bd_ini_externa.head()

,_id_,comp_externo1,comp_externo2,comp_externo3,comp_externo4
0,78213,260.869565,25.854639,56.0,-230.0
1,76457,40.579710,2.872738,62.0,-230.0
2,40666,NaN,NaN,119.0,-230.0
3,8329,349.275362,34.616490,94.0,-230.0
4,68389,276.811594,27.434645,125.0,-230.0


In [4]:
bd_ini_externa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   _id_           50000 non-null  int64  
 1   comp_externo1  47831 non-null  float64
 2   comp_externo2  44410 non-null  float64
 3   comp_externo3  49994 non-null  float64
 4   comp_externo4  50000 non-null  float64
dtypes: float64(4), int64(1)
memory usage: 1.9 MB


In [5]:
len(bd_ini_externa["_id_"].unique())

50000

Podemos observar que en esta base no hay identificadores nulos y además el número de identificadores distintos es el total de identificadores, por lo que no hay identificadores repetidos.

Sin embargo, los componentes externos 1, 2 y 3 sí tienen 2169, 5590 y 6 celdas nulas respectivamente.
**NOTA: ¿Tenemos que cambiar los null?**

In [6]:
bd="base_ini_interna.csv"
bd_ini_interna = pd.read_csv(ruta+bd)
bd_ini_interna.head()

,_id_,fecha_inicio,comp_interno1,comp_interno2,comp_interno3,comp_interno4,categoria1,categoria2,categoria3
0,78213,2020-03-25,0.000000,b'0',11.0,58.0,2.0,9.0,156.0
1,76457,2020-02-13,0.000000,b'1',28.0,45.0,1.0,4.0,173.0
2,40666,2018-01-24,0.000000,b'1',25.0,45.0,1.0,9.0,170.0
3,8329,2016-10-07,0.000000,b'1',14.0,59.0,3.0,3.0,159.0
4,68389,2019-07-30,2026.342452,b'1',1.0,67.0,2.0,9.0,146.0


In [7]:
bd_ini_interna.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   _id_           50000 non-null  int64  
 1   fecha_inicio   50000 non-null  object 
 2   comp_interno1  50000 non-null  float64
 3   comp_interno2  50000 non-null  object 
 4   comp_interno3  50000 non-null  float64
 5   comp_interno4  50000 non-null  float64
 6   categoria1     50000 non-null  float64
 7   categoria2     50000 non-null  float64
 8   categoria3     50000 non-null  float64
dtypes: float64(6), int64(1), object(2)
memory usage: 3.4+ MB


In [8]:
len(bd_ini_interna["_id_"].unique())

50000

Por lo que esta base no tiene entradas nulas, tiene el mismo número de identificadores que la base anterior y todos son distintos.

In [9]:
bd="base_ini_parte1.csv"
bd_ini_parte1 = pd.read_csv(ruta+bd)
bd_ini_parte1.head()

,_id_,_id2_,comp_mixto1
0,78213,1798899,0.33522
1,76457,1758511,0.33334
2,40666,935318,0.29186
3,8329,191567,0.29112
4,68389,1572947,0.27318


In [10]:
bd_ini_parte1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   _id_         50000 non-null  int64  
 1   _id2_        50000 non-null  int64  
 2   comp_mixto1  50000 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 1.1 MB


In [11]:
len(bd_ini_parte1["_id_"])

50000

In [12]:
len(bd_ini_parte1["_id2_"])

50000

Por lo tanto en esta base no hay entradas con _null_ y ambos identificadores son únicos.

In [13]:
bd = "base_comportamiento_2.txt"
bd_comportamiento_2 = pd.read_csv(ruta+bd,sep=" ",low_memory=False)
bd_comportamiento_2.head()

,num_caso,mes_informacion,Tiempo_aparicion
0,1,2016-04-01 00:00:00,0.0
1,1,2016-05-01 00:00:00,1.0
2,1,2016-06-01 00:00:00,2.0
3,1,2016-07-01 00:00:00,3.0
4,1,2016-08-01 00:00:00,4.0


In [14]:
bd_comportamiento_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1455536 entries, 0 to 1455535
Data columns (total 3 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   num_caso          1455536 non-null  int64  
 1   mes_informacion   1455536 non-null  object 
 2   Tiempo_aparicion  1455536 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 33.3+ MB


In [15]:
len(bd_comportamiento_2["num_caso"].unique())

49352

Podemos observar que todas las llaves, en este caso _num_caso_ son no vacías. Por ejemplo, esta base (_bd_comportamiento_2_) tiene un comportamiento peculiar. Parece ser que, en resumen, registra el tiempo desde que un caso (_id_) empieza un trámite. Sin embargo, existe una duda: ¿qué pasa si un caso inicia otro procedimiento en algún momento dado? ¿se vuelve a comenzar desde cero? La siguiente consulta está pensada para que nos dé esa información, además, en el proceso, se logra que sólo aparezcan los valores únicos de los números de casos:

In [30]:
query ="""
       select num_caso,count(num_caso), max(Tiempo_aparicion) 
       from bd_comportamiento_2 group by num_caso       
       """
sqldf.run(query).head()

,num_caso,count(num_caso),max(Tiempo_aparicion)
0,1,62,61.0
1,7,29,28.0
2,8,63,51.0
3,9,15,14.0
4,10,16,15.0
5,14,56,55.0
6,18,58,57.0
7,21,54,53.0
8,22,26,25.0
9,24,4,3.0


La consulta en el registro 2 indica que el máximo registro del tiempo no siempre es lo mismo que la cantidad de registros que tiene un caso dado (salvo por un número que se agrega por el cero) que es lo que se esperaría. Esto es un indicativo de que , en efecto, la base puede iniciar el registro de un mismo caso más adelante en el tiempo. La siguiente consulta dice cuántas veces un caso inició un proceso:

In [31]:
query ="""
       select num_caso,count(Tiempo_aparicion) as cantidad_tramites 
       from bd_comportamiento_2 where Tiempo_aparicion=0
       group by num_caso
       """
no_tramites=sqldf.run(query)

In [34]:
query ="""
       select num_caso,max(mes_informacion) as ultima_fecha 
       from bd_comportamiento_2
       group by num_caso
       """
ultimafecha=sqldf.run(query)

In [39]:
query ="""
       select n.num_caso,n.cantidad_tramites,u.ultima_fecha
       from no_tramites as n inner join ultimafecha as u on n.num_caso=u.num_caso
       """
sintesis_c2=sqldf.run(query)

In [54]:
sintesis_c2.head(10)

,num_caso,cantidad_tramites,ultima_fecha
0,1,1,2021-05-01 00:00:00
1,7,1,2018-05-01 00:00:00
2,8,5,2021-04-01 00:00:00
3,9,1,2017-04-01 00:00:00
4,10,1,2017-05-01 00:00:00
5,14,1,2020-09-01 00:00:00
6,18,1,2021-05-01 00:00:00
7,21,1,2021-05-01 00:00:00
8,22,1,2018-12-01 00:00:00
9,24,1,2016-11-01 00:00:00


In [45]:
bd_comportamiento_2.groupby("num_caso")["mes_informacion"].value_counts().to_frame()

mes_informacion
num_caso mes_informacion                     
1        2016-04-01 00:00:00                1
         2016-05-01 00:00:00                1
         2016-06-01 00:00:00                1
         2016-07-01 00:00:00                1
         2016-08-01 00:00:00                1
...                                       ...
98990    2021-05-01 00:00:00                1
98992    2021-04-01 00:00:00                1
         2021-05-01 00:00:00                1
98997    2021-04-01 00:00:00                1
         2021-05-01 00:00:00                1

[1455536 rows x 1 columns]

In [53]:
query="""
      select num_caso,mes_informacion 
      from bd_comportamiento_2 
      group by num_caso,mes_informacion 
      having count(mes_informacion)>=2
"""
sqldf.run(query)

,num_caso,mes_informacion


In [67]:
bd = "base_comportamiento_3.csv"
bd_comportamiento_3 = pd.read_csv(ruta+bd,low_memory=False)
bd_comportamiento_3.head(15)

,num_caso,mes_informacion,comportamiento_j
0,1,2016-04-01 00:00:00,0.0
1,1,2016-05-01 00:00:00,0.0
2,1,2016-06-01 00:00:00,0.0
3,1,2016-07-01 00:00:00,0.0
4,1,2016-08-01 00:00:00,0.0
5,1,2016-09-01 00:00:00,0.0
6,1,2016-10-01 00:00:00,0.0
7,1,2016-11-01 00:00:00,0.0
8,1,2016-12-01 00:00:00,0.0
9,1,2017-01-01 00:00:00,1.0


In [19]:
bd_comportamiento_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1455536 entries, 0 to 1455535
Data columns (total 3 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   num_caso          1455536 non-null  int64  
 1   mes_informacion   1455536 non-null  object 
 2   comportamiento_j  1455536 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 33.3+ MB


In [58]:
query="""
      select num_caso, count(comportamiento_j) 
      from bd_comportamiento_3
      where comportamiento_j!=0
      group by num_caso
"""
sqldf.run(query).head(20)

,num_caso,count(comportamiento_j)
0,1,7
1,7,9
2,8,4
3,10,1
4,14,7
5,18,8
6,21,3
7,22,2
8,25,1
9,27,9


In [60]:
query="""
      select num_caso, avg(comportamiento_j) 
      from bd_comportamiento_3
      group by num_caso
"""
sqldf.run(query).head(20)

,num_caso,avg(comportamiento_j)
0,1,0.112903
1,7,0.827586
2,8,0.063492
3,9,0.000000
4,10,0.062500
5,14,0.125000
6,18,0.137931
7,21,0.055556
8,22,0.076923
9,24,0.000000


In [62]:
query="""select num_caso,min(mes_informacion),max(mes_informacion)
         from bd_comportamiento_3
         group by num_caso"""
sqldf.run(query).head()

,num_caso,min(mes_informacion),max(mes_informacion)
0,1,2016-04-01 00:00:00,2021-05-01 00:00:00
1,7,2016-01-01 00:00:00,2018-05-01 00:00:00
2,8,2016-02-01 00:00:00,2021-04-01 00:00:00
3,9,2016-02-01 00:00:00,2017-04-01 00:00:00
4,10,2016-02-01 00:00:00,2017-05-01 00:00:00


In [78]:
query1="""select 
         num_caso,
         mes_informacion,
         comportamiento_j,
         max(mes_informacion)-mes_informacion
         from bd_comportamiento_3
         group by num_caso
         """
sqldf.run(query1).head(20)

,num_caso,mes_informacion,comportamiento_j,max(mes_informacion)-mes_informacion
0,1,2021-05-01 00:00:00,0.0,0
1,7,2018-05-01 00:00:00,0.0,0
2,8,2021-04-01 00:00:00,0.0,0
3,9,2017-04-01 00:00:00,0.0,0
4,10,2017-05-01 00:00:00,0.0,0
5,14,2020-09-01 00:00:00,0.0,0
6,18,2021-05-01 00:00:00,0.0,0
7,21,2021-05-01 00:00:00,0.0,0
8,22,2018-12-01 00:00:00,0.0,0
9,24,2016-11-01 00:00:00,0.0,0


In [20]:
len(bd_comportamiento_3["num_caso"].unique())

49352

**REVISAR**

In [21]:
bd="base_inversion.txt"
bd_base_inversion= pd.read_csv(ruta+bd,sep=" ")
bd_base_inversion.head()

,_id2_,inversion
0,1758511,0.041039
1,1572947,77.422135
2,1051836,90.539953
3,1223255,90.205018
4,1894602,1.071354


In [22]:
bd_base_inversion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17536 entries, 0 to 17535
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   _id2_      17536 non-null  int64  
 1   inversion  17536 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 274.1 KB


In [23]:
len(bd_base_inversion["_id2_"].unique())

17536

In [24]:
bd="base_ini_parte1.csv"
bd_ini_parte1= pd.read_csv(ruta+bd)
bd_ini_parte1.head()

,_id_,_id2_,comp_mixto1
0,78213,1798899,0.33522
1,76457,1758511,0.33334
2,40666,935318,0.29186
3,8329,191567,0.29112
4,68389,1572947,0.27318


In [25]:
bd_ini_parte1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   _id_         50000 non-null  int64  
 1   _id2_        50000 non-null  int64  
 2   comp_mixto1  50000 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 1.1 MB


In [26]:
len(bd_ini_parte1["_id_"].unique())

50000

In [27]:
len(bd_ini_parte1["_id2_"].unique())

50000

### Parte 2: Cruces
¿Cómo hicimos los cruces?

### Parte 3: Análisis
¿Qué análisis podemos realizar?